# Setup

Import the required modules. Note that these are for the whole course; some of them may not be required for this specific lecture.

In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn.linear_model
import sklearn.neighbors
import sklearn.neural_network

import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing

Initialize the random seed, for reproducibility, so we all get the same results

In [ ]:
np.random.seed(42)

## Data directory and Google Colab

I write and run notebooks on Google Colab with data on Google Drive. This cell checks if the notebook is running on Google Colab. If so, it connects to Google Drive. Otherwise, it will look for the data in the current directory.

In [ ]:
import sys
RUNNING_ON_COLAB = 'google.colab' in sys.modules

if RUNNING_ON_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  DIR = "drive/MyDrive"
else:
  DIR = "."

# Comparison of 3 classifiers with credit card transactions

We have learned three classifiers in class:

1. k-Nearest Neighbors (with majority voting, i.e. the prediction is the most common class among the k nearest neighbors);

2. logistic regression (binomial / binary, or multinomial / multi-class);

3. neural networks, or the multi-layer perceptron.

Here, we estimate these three classifiers on a dataset of credit card transactions.

## Data

We use a publicly available dataset on [credit card transactions](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud). For privacy reasons, the explanatory variables (`V1-V28`) in this dataset are obfuscated (they come from "dimensionality reduction" via Principal Components Analysis, or PCA, and these are the most relevant factors). The other variables are the transaction amount and the label (1 for fraud, 0 for non-fraud). You need to unzip and extract the dataset in `creditcardfraud.zip`.

In [ ]:
DATA_FILEPATH = os.path.join(DIR, "creditcard.csv")
df = pd.read_csv(DATA_FILEPATH)

## Summary statistics

When we get a new dataset, the first thing we should do is look at a few samples of the data. We do this in Pandas with `.head()`, a method on dataframes.

In [ ]:
df.head()

We see that we have variables called:

- `Time` (instead of a timestamp, which could be a privacy risk, this is the time elapsed since the first transaction);

- a number of variables, `V1-V28` that look distributed around 0 with a standard deviation around 1;

- `Amount`, which looks like a dollar amount in cents;

- `Class`, which in this case is full of zeroes.

The next step is to print summary statistics, with Pandas's dataframe method `.describe()`:

In [ ]:
df.describe()

We see that:

- we have around 300 thousand observations;

- the variables `V1-V28` are indeed centered around 0 with a standard deviation close to unity;

- the average purchase amount is 88 dollars, ranging from 0 (for example, for a card verification in a free trial, that does not charge money at the start) to 26 thousand dollars (for example, to buy a car)

- `Class`, which is the flag for a fraudulent transaction, is mostly zeroes, with 0.17% of transactions that are fraudulent.

This is called "class imbalance": one class (normal transactions) covers most of the dataset. This will become a problem: a prediction function that returns 0 in all cases will on average achieve 99.83% accuracy (100% - 0.17%).

## Plot the data

The second thing to do with a new dataset is to plot the data. Here, we plot two overlaying histograms of each of the 28 variables, once for fraudulent transactions, and again for legitimate transactions. We scale the histograms so that we see the distribution depsite the class imbalance.

We access the rows of the data

In [ ]:
y = df["Class"]

legitimate = df[y == 0]
fraudulent = df[y == 1]

for num in range(1, 29):
  variable = f"V{num:d}"

  min_x = min(df[variable])
  max_x = max(df[variable])

  bins = np.linspace(min_x, max_x, 100)

  plt.hist(legitimate[variable], bins, alpha=0.5, label="legit", density=True)
  plt.hist(fraudulent[variable], bins, alpha=0.5, label="fraud", density=True)
  plt.title(variable)
  plt.legend(loc="upper right")
  plt.show()

Some of these have clearly different distributions, so we should find a model with reasonable accuracy in distinguishing legitimate and fraudulent transactions.

We also see that some distributions have wide stretches, i.e. there are outliers. For example, for `V28`, the values are centered around 0 with 1-2 for standard deviation, but the maximum value is over 30.

In a real-world application, you may want to deal with these outliers first, for example:

- discarding outliers;

- "winsorizing" extreme values (so the maximum value of 30 gets replaced with the value at the 95% quantile, for example);

- choosing a model that is not sensitive to outliers.

In this course, for simplicity, we ignore them.

## Hyperparameters

We have already trained a neural network before, i.e. we found the optimal parameters (weights and biases) to minimize the cross-entropy loss.

There are other parameters we can optimize to improve the model: the number of layers, the number of neurons in each layer, the activation function in each layer, etc. To distinguish these from the usual term "parameters" in statistics, we call these "hyperparameters".

## Hyperparameter optimization

These parameters can be optimized, similar to parameters. For example, take the activation function. A model using logistic/sigmoid is different from a model using ReLU. We choose the best between the two by comparing some criterion (cross-entropy loss, or accuracy) on data that neither model has seen before.

It is similar to kNN and the choice of k. In the kNN example, we split a dataset into training and testing. The training dataset serves to "train the model" (even though kNN has no training). The test dataset served to optimize the value of k using the accuracy of the model on data that the model has not seen (otherwise, it would be similar to giving an exam with exercises from assignments or lectures: students could just repeat what they already saw).

The dataset that serves for hyperparameter optimization is called "validation dataset".

The procedure is then:

- one or more for loops on the hyperparameter values we want to optimize;

- for each possible combination, train the model on the training dataset;

- compute a criterion (cross-entropy loss or accuracy) of the trained model on a test dataset;

- choose the value of the hyperparameters that optimizes that criterion.

## Train-validation-test split

We can use the training and validation datasets to optimize one particular model, such as neural networks. To compare across models, such as neural networks versus kNN, we need another dataset, that no model has seen before. This is typically called the "test dataset".

So our original dataset gets split into 3 datasets, called  a "train-validation-test" split:

- the training data serves to "train the model", e.g. to train logistic regression, kNN or a neural network;

- the validation dataset serves for hyperparameter optimization: finding the best value of k (in the case of kNN), or to find the best activation function, or number of layers, or number of neurons in each layer (in the case of neural networks);

- the test dataset serves to compare the accuracy across models and choose the best one, on data that no model has seen before.

Since logistic regression does not allow hyperparameter optimization, we can actually estimate it on the combination of training and validation datasets.

## Train-test-validation split in SKLearn

SciKit-Learn has no possibility to split a dataset into three parts, so we split into two parts twice. We want a split 60-20-20, so we split first at 20%, then at 25% (because 25% of (100 - 80) = 0.25 * 0.8 = 20%).

In [ ]:
from sklearn.model_selection import train_test_split

y = df["Class"]

variables = list([f"V{num:d}" for d in range(1, 29)])
X = df[variables]

train_test_split(X, y, random_state=0, test_size=0.7)

X_train_val, X_test, y_train_val, y_test = train_test_split(X,
                                                            y,
                                                            random_state=42,
                                                            test_size=0.2)

X_train, X_val, y_train, y_val = train_test_split(X_train_val,
                                                  y_train_val,
                                                  random_state=42,
                                                  test_size=0.25)

print(f"{X_train.shape=}")
print(f"{X_val.shape=}")
print(f"{X_test.shape=}")

## Logistic regression / classification

Let's estimate a logistic regression on the training + validation dataset:

In [ ]:
logistic_classifier = sklearn.linear_model.LogisticRegression(max_iter=int(1e5))
logistic_classifier.fit(X_train_val, y_train_val)

## k-Nearest Neighbors

Let's optimize the k-Nearest Neighbors classifier for k between, say, 1 and 10 thousand (10 thousand so that the classifier has a chance to find some fraudulent transactions, on average, 17 of them, since the proportion of fraudulent transactions is 0.17%).

This would take a very long time: for one value of k, the prediction on the test set compares all elements in the test set to all elements in the training set, so around 10 billion (170,883 * 56,962 = 9,733,837,446) computation of Euclidean distance in 28 dimensions.

Instead, let's do it with a logarithmic scale: 1, 10, 100, 1000, 10000:

In [ ]:
best_accuracy = 0
best_k = 0
best_knn = None

for k in [10, 100, 1000, 10000]:

  print(f"Trying {k=}... ", end="")
  knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=k)
  knn.fit(X_train, y_train)

  print("fit! Computing accuracy... ", end="")
  accuracy = knn.score(X_val, y_test)

  print(f"computed! {accuracy=:.6f}")
  if accuracy > best_accuracy:
    best_k = k
    best_knn = knn
    best_accuracy = accuracy

print(f"The optimal value of k is {best_k} with accuracy {best_accuracy:.2f}")

## Neural networks

We could try a neural network with between 1 and 3 layers, with between 10 and 20 neurons each, and with activation functions either logistic or ReLU. But that would also take too long. So instead, let's try 2-3 hidden layers, with a number of neurons either 10, 15 or 20, but strictly decreasing from one layer to the next. The list of possible hidden layer sizes is then:

```
[(20, 15), (20, 10), (15, 10), (20, 15, 10)]
```

Next, we optimize over number of layers, neurons in each layer, and activation function to find the best fit of a Multi-Layer Perceptron:

In [ ]:
best_accuracy = 0
best_hidden_layers = None
best_mlp = None
best_activation = None

for combination in [(20, 15), (20, 10), (15, 10), (20, 15, 10)]:
  for activation in ["logistic", "relu"]:

    print(f"Fitting {combination=}, {activation=}... ", end="")
    mlp = sklearn.neural_network.MLPClassifier(
      hidden_layer_sizes=combination,
      max_iter=2000,
      random_state=42,
      activation=activation
    )
    mlp.fit(X_train, y_train)

    print("Fit! Accuracy... ", end="")
    y_predicted = mlp.predict(X_val)
    accuracy = sklearn.metrics.accuracy_score(y_predicted, y_val)
    # Alternatively:
    # accuracy = mlp.score(X_val, y_val)

    print(f"computed! {accuracy=:.6f}")

    if accuracy > best_accuracy:
      best_accuracy = accuracy
      best_hidden_layers = combination
      best_mlp = mlp
      best_activation = activation

print(f"Found best MLP model: {best_hidden_layers=}, {best_activation=}")
print(f"{best_accuracy=}")

## Comparison of the three classifiers

Now that we have estimated and fit the three classifiers, let's compare their performance on the test dataset.

In [ ]:
acc_test_logistic = logistic_classifier.score(X_test, y_test)

acc_test_knn = best_knn.score(X_test, y_test)

acc_test_mlp = best_mlp.score(X_test, y_test)

print(f"{acc_test_logistic=}, {acc_test_knn=}, {acc_test_mlp=}")

Note: it's possible to make this code DRY with the function `eval()`, which converts a string into code and evaluates that code. The following cell does the same as the previous, but is shorter, and more complicated (and also uses the paradigm of "functions as first class citizens", storing a function in a variable, and using it after). Try this only if you're comfortable with the material so far. It will not be in the exam:

In [ ]:
for model_string in ["logistic_classifier", "best_knn", "best_mlp"]:
  model_function = eval(model_string)
  accuracy = model_fn.score(X_test, y_test)
  print(f"For model {model_string}, {accuracy=:.6f}")

The accuracy scores are similar for all models, due to the class imbalance (read more [here](https://stackoverflow.com/questions/67878862/why-are-all-my-classification-accuracy-scores-the-same)).

## Situation specific to the class imbalance

Without the class imbalance, the procedure ends here: we choose the best model from a criterion of accuracy.

When the data has a class imbalance like this credit card dataset, we may need more. For example, a naive model that always predict a legitimate transaction achieves 99.83% accuracy.

We could pursue the analysis with other metrics, for example the F-1 score, plotting a confusion matrix, or the Area Under the Precision-Recall Curve (AUPRC).